In [48]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 
import datetime
from imps_zda import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
#input train data from csv
from proj1_helpers import *
DATA_TRAIN_PATH = 'D:\\Jupyter Notebook\Machine Learning\project1\data\\train.csv'  
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [50]:
#check the shape of y,tX,ids
y.shape,tX.shape,ids.shape

((250000,), (250000, 30), (250000,))

In [51]:
tX = np.delete(tX,np.array(range(0,13)),1)

array([2., 1., 1., ..., 1., 0., 0.])

In [5]:
import pandas as pd

dataset= pd.DataFrame(tX)

In [9]:
corr = dataset.corr()


In [16]:
print(corr)

          0         1         2         3         4         5         6   \
0   1.000000 -0.455755  0.168548  0.198765  0.162661  0.160524  0.162521   
1  -0.455755  1.000000  0.190109 -0.249116 -0.176386 -0.190392 -0.175942   
2   0.168548  0.190109  1.000000 -0.062562 -0.032251 -0.040620 -0.032126   
3   0.198765 -0.249116 -0.062562  1.000000  0.523664  0.534531  0.523639   
4   0.162661 -0.176386 -0.032251  0.523664  1.000000  0.946045  0.999981   
5   0.160524 -0.190392 -0.040620  0.534531  0.946045  1.000000  0.944443   
6   0.162521 -0.175942 -0.032126  0.523639  0.999981  0.944443  1.000000   
7   0.228105  0.043251  0.579712 -0.539379 -0.299076 -0.303751 -0.299083   
8   0.045826  0.017758 -0.000702  0.310501  0.270878  0.247771  0.271435   
9   0.201464 -0.146837  0.088685  0.832733  0.671996  0.680700  0.672158   
10 -0.017073  0.349504  0.097490  0.089187  0.041307  0.027674  0.041614   
11  0.358963 -0.419757 -0.090846  0.539356  0.368492  0.369803  0.368177   
12  0.162623

In [23]:
np.corrcoef(tX[:,2],tX[:,0])

array([[1.        , 0.16854799],
       [0.16854799, 1.        ]])

In [17]:
def data_correlation(tX):
    dim = tX.shape[1]
    corr = np.zeros((dim,dim))
    
    for t in range(dim):
        for k in range(dim):
            corr[t,k] = np.corrcoef(tX[:,t],tX[:,k])[0,1]
    
    return corr

In [68]:
#import the name of each feature
DER_mass_MMC = tX[:,0]
DER_mass_transverse_met_lep=tX[:,1]
DER_mass_vis = tX[:,2]
DER_pt_h = tX[:,3]
DER_deltaeta_jet_jet = tX[:,4]
DER_mass_jet_jet = tX[:,5]
DER_prodeta_jet_jet = tX[:,6]
DER_deltar_tau_lep = tX[:,7]
DER_pt_tot = tX[:,8]
DER_sum_pt = tX[:,9]
DER_pt_ratio_lep_tau = tX[:,10]
DER_met_phi_centrality = tX[:,11]
DER_lep_eta_centrality = tX[:,12]
PRI_tau_pt = tX[:,13]
PRI_tau_eta = tX[:,14]
PRI_tau_phi = tX[:,15]
PRI_lep_pt = tX[:,16]
PRI_lep_eta =tX[:,17]
PRI_lep_phi = tX[:,18]
PRI_met=tX[:,19]
PRI_met_phi = tX[:,20]
PRI_met_sumet = tX[:,21]
PRI_jet_num=tX[:,22]
PRI_jet_leading_pt=tX[:,23]
PRI_jet_leading_eta = tX[:,24]
PRI_jet_leading_phi = tX[:,25]
PRI_jet_subleading_pt=tX[:,26]
PRI_jet_subleading_eta = tX[:,27]
PRI_jet_subleading_phi = tX[:,28]
PRI_jet_all_pt = tX[:,29]

In [69]:
#do some PCA

In [70]:
def p_x(p_t,phi):
    
    length = len(p_t)
    px = np.zeros(length)
    valid_index = (p_t!=-999)
    
    px[valid_index] = p_t[valid_index]*np.cos(phi[valid_index])
    px[~valid_index] = -999
        
    return px

def p_y(p_t,phi):

    length = len(p_t)
    py = np.zeros(length)
    valid_index = (p_t!=-999)
    
    py[valid_index] = p_t[valid_index]*np.sin(phi[valid_index])
    py[~valid_index] = -999
    
    return py

def p_z(p_t,eta):

    length = len(p_t)
    pz = np.zeros(length)
    valid_index = (p_t!=-999)
    
    pz[valid_index] = p_t[valid_index]*np.sinh(eta[valid_index])
    pz[~valid_index] = -999
    
    return pz

#mass are neglected, E = p
def particle_energy(px,py,pz):
    
    length = len(px)
    energy = np.zeros(length)
    valid_index = (px!=-999)
    
    energy[valid_index] = np.sqrt(px[valid_index]**2+py[valid_index]**2+pz[valid_index]**2)
    energy[~valid_index] = -999
    return energy

def cross_product(px_1,py_1,pz_1,px_2,py_2,pz_2):
    N = len(px_1)
    cp_x = np.zeros(N)
    cp_y = np.zeros(N)
    cp_z = np.zeros(N)
    for t in range(N):
        if(px_1[t]!=-999 and px_2[t]!=-999):
            temp_cross=np.cross(np.array([px_1[t],py_1[t],pz_1[t]]),np.array([px_2[t],py_2[t],pz_2[t]]))
            cp_x[t] = temp_cross[0]
            cp_y[t] = temp_cross[1]
            cp_z[t] = temp_cross[2]
        else:
            cp_x[t] = -999
            cp_y[t] = -999
            cp_z[t] = -999
    return cp_x,cp_y,cp_z

def dot_product(px_1,py_1,pz_1,px_2,py_2,pz_2):
    N = len(px_1)
    dp = np.zeros(N)
    for t in range(N):
        if(px_1[t]!=-999 and px_2[t]!=-999):
            dp[t] = np.inner(np.array([px_1[t],py_1[t],pz_1[t]]),np.array([px_2[t],py_2[t],pz_2[t]]))
        else:
            dp[t] = -999
    return dp

def cosine_similarity(px_1,py_1,pz_1,px_2,py_2,pz_2):
    length = len(px_1)
    
    cp = np.zeros(length)
    
    valid_index = (px_1!=-999)&(px_2!=-999)
    

    cp[valid_index] = dot_product(px_1[valid_index],py_1[valid_index],pz_1[valid_index],px_2[valid_index],py_2[valid_index],pz_2[valid_index])/(np.sqrt(px_1[valid_index]**2+py_1[valid_index]**2+pz_1[valid_index]**2)*np.sqrt(px_2[valid_index]**2+py_2[valid_index]**2+pz_2[valid_index]**2))
    cp[~valid_index] = -999
    
    return cp

def determinant_vector(px_1,py_1,pz_1,px_2,py_2,pz_2,px_3,py_3,pz_3):
    N = len(px_1)
    dv = np.zeros(N)
    
    for t in range(N):
        if((px_1[t]!=-999)and(px_2[t]!=-999)and(px_3[t]!=-999)):
            temp_vector = np.array([[px_1[t],py_1[t],pz_1[t]],[px_2[t],py_2[t],pz_2[t]],[px_3[t],py_3[t],pz_3[t]]])
            dv[t] = np.linalg.det(temp_vector)
        else:
            dv[t] = -999;

    return dv

def sum_p_xyz(px,py,pz):
    
    length = len(px)
    sp=np.zeros(length)
    valid_index = (px!=-999)
    sp[valid_index] = px[valid_index]+py[valid_index]+pz[valid_index]
    sp[~valid_index] = -999
    return sp

#Compute Transverse Mass 
def transverse_mass(a_t,a_phi,b_t,b_phi):
    
    length = len(a_t)
    mass = np.zeros(length)
    valid_index = (a_t!=-999)&(b_t!=-999)
    
    mass[valid_index] = np.sqrt((a_t[valid_index]+b_t[valid_index])**2-(a_t[valid_index]*np.cos(a_phi[valid_index])+b_t[valid_index]*np.cos(b_phi[valid_index]))**2-(a_t[valid_index]*np.sin(a_phi[valid_index])+b_t[valid_index]*np.sin(b_phi[valid_index]))**2)
    
    mass[valid_index] = check_transverse_mass(mass[valid_index])
    mass[~valid_index] = -999
    
    return mass

def check_transverse_mass(mass):
    length = len(mass)
    
    median = np.median(mass[np.isnan(mass)==0])
    for t in range(length):
        if(np.isnan(mass[t])==1):
            mass[t] = median
    return mass

def invariant_mass(a_t,a_eta,a_phi,b_t,b_eta,b_phi):
    length = len(a_t)
    
    mass = np.zeros(length)
    
    valid_index = (a_t!=-999)&(b_t!=-999)
    
    
    a_z = a_t[valid_index]*np.sinh(a_eta[valid_index])
    b_z = b_t[valid_index]*np.sinh(b_eta[valid_index])
    
    a_xyz = np.sqrt(a_t[valid_index]**2+(a_z)**2)
    b_xyz = np.sqrt(b_t[valid_index]**2+(b_z)**2)
    ab_x = a_t[valid_index]*np.cos(a_phi[valid_index])+b_t[valid_index]*np.cos(b_phi[valid_index])
    ab_y =a_t[valid_index]*np.sin(a_phi[valid_index])+b_t[valid_index]*np.sin(b_phi[valid_index])
    ab_z =a_z+b_z
    
    
    mass[valid_index] = np.sqrt((a_xyz+b_xyz)**2-(ab_x)**2-(ab_y)**2-(ab_z)**2)
    mass[~valid_index] = -999
    
    return mass

def phi_minus_phi(a_phi,b_phi):
    
    length = len(a_phi)
    result = np.zeros(length)
    
    valid_index = (a_phi!=-999)&(b_phi!=-999)
    
    result[valid_index] = a_phi[valid_index]-b_phi[valid_index]
    result[~valid_index] = -999
    
    return result

In [36]:
#generate px,py,pz and append to the tX
#tau
p_tau_x = p_x(PRI_tau_pt,PRI_tau_phi)
p_tau_y = p_y(PRI_tau_pt,PRI_tau_phi)
p_tau_z = p_z(PRI_tau_pt,PRI_tau_eta)
#lep
p_lep_x = p_x(PRI_lep_pt,PRI_lep_phi)
p_lep_y = p_y(PRI_lep_pt,PRI_lep_phi)
p_lep_z = p_z(PRI_lep_pt,PRI_lep_eta)
#met
p_met_x = p_x(PRI_met,PRI_met_phi)
p_met_y = p_y(PRI_met,PRI_met_phi)

#append features
tX=np.append(tX,np.array([p_tau_x,p_tau_y,p_tau_z,p_lep_x,p_lep_y,p_lep_z,p_met_x,p_met_y]).T,axis=1)

#jet leading and subleading x y z

jet_leading_p_x = p_x(PRI_jet_leading_pt,PRI_jet_leading_phi)
jet_leading_p_y = p_y(PRI_jet_leading_pt,PRI_jet_leading_phi)
jet_leading_p_z = p_z(PRI_jet_leading_pt,PRI_jet_leading_eta)

jet_subleading_p_x = p_x(PRI_jet_subleading_pt,PRI_jet_subleading_phi)
jet_subleading_p_y = p_y(PRI_jet_subleading_pt,PRI_jet_subleading_phi)
jet_subleading_p_z = p_z(PRI_jet_subleading_pt,PRI_jet_subleading_eta)

tX=np.append(tX,np.array([jet_leading_p_x,jet_leading_p_y,jet_leading_p_z]).T,axis=1)
tX=np.append(tX,np.array([jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z]).T,axis=1)

In [37]:
#generate energy of particle
p_tau_energy = particle_energy(p_tau_x,p_tau_y,p_tau_z)
p_lep_energy = particle_energy(p_lep_x,p_lep_y,p_lep_z)

#append features
tX=np.append(tX,np.array([p_tau_energy,p_lep_energy]).T,axis=1)

#energy of jet leading and subleading
jet_leading_p_energy = particle_energy(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
jet_subleading_p_energy = particle_energy(jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)

tX=np.append(tX,np.array([jet_leading_p_energy,jet_subleading_p_energy]).T,axis=1)

In [38]:
#transform phi
lep_tau_phi = phi_minus_phi(PRI_lep_phi,PRI_tau_phi)
met_tau_phi = phi_minus_phi(PRI_met_phi,PRI_tau_phi)
jet_tau_phi = phi_minus_phi(PRI_jet_leading_phi,PRI_tau_phi)
jet_sub_tau_phi = phi_minus_phi(PRI_jet_subleading_phi,PRI_tau_phi)

tX=np.append(tX,np.array([lep_tau_phi,met_tau_phi,jet_tau_phi,jet_sub_tau_phi]).T,axis=1)

In [39]:
N = len(y)
#sum of px,py,pz
tau_sum = sum_p_xyz(p_tau_x,p_tau_y,p_tau_z)
lep_sum = sum_p_xyz(p_lep_x,p_lep_y,p_lep_z)
met_sum = sum_p_xyz(p_met_x,p_met_y,np.zeros(N))
jet_leading_sum = sum_p_xyz(jet_leading_p_x,jet_leading_p_y,jet_leading_p_z)
jet_subleading_sum = sum_p_xyz(jet_subleading_p_x,jet_subleading_p_y,jet_subleading_p_z)
#append features
tX=np.append(tX,np.array([tau_sum,lep_sum,met_sum,jet_leading_sum,jet_subleading_sum]).T,axis=1)

In [40]:
#transverse mass
transverse_mass_tau_lep = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_lep_pt,PRI_lep_phi)
transverse_mass_tau_met = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_met,PRI_met_phi)
transverse_mass_tau_jet = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_tau_jet_sub = transverse_mass(PRI_tau_pt,PRI_tau_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
transverse_mass_lep_jet = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_lep_jet_sub = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
#transverse_mass_lep_met = transverse_mass(PRI_lep_pt,PRI_lep_phi,PRI_met,PRI_met_phi) already have
transverse_mass_met_jet = transverse_mass(PRI_met,PRI_met_phi,PRI_jet_leading_pt,PRI_jet_leading_phi)
transverse_mass_met_jet_sub= transverse_mass(PRI_met,PRI_met_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi)
#transverse_mass_jet_jet_sub= transverse_mass(PRI_jet_leading_pt,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_phi) already have
#append features
tX=np.append(tX,np.array([transverse_mass_tau_lep,transverse_mass_tau_met,transverse_mass_tau_jet,transverse_mass_tau_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([transverse_mass_lep_jet,transverse_mass_lep_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([transverse_mass_met_jet,transverse_mass_met_jet_sub]).T,axis=1)

C:\DEVELOPMENT\anconda3\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in sqrt


In [41]:
#invariant mass
#already have invariant mass between tau and lep and between two jets
invariant_mass_tau_met =invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_met,np.zeros(N),PRI_met_phi)
invariant_mass_lep_met = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_met,np.zeros(N),PRI_met_phi)
invariant_mass_tau_jet = invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_tau_jet_sub = invariant_mass(PRI_tau_pt,PRI_tau_eta,PRI_tau_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)
invariant_mass_lep_jet = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_lep_jet_sub = invariant_mass(PRI_lep_pt,PRI_lep_eta,PRI_lep_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)
invariant_mass_met_jet =invariant_mass(PRI_met,np.zeros(N),PRI_met_phi,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi)
invariant_mass_met_jet_sub = invariant_mass(PRI_met,np.zeros(N),PRI_met_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi)



#append features
tX=np.append(tX,np.array([invariant_mass_tau_met,invariant_mass_lep_met]).T,axis=1)
tX=np.append(tX,np.array([invariant_mass_tau_jet,invariant_mass_tau_jet_sub,invariant_mass_lep_jet,invariant_mass_lep_jet_sub]).T,axis=1)
tX=np.append(tX,np.array([invariant_mass_met_jet,invariant_mass_met_jet_sub]).T,axis=1)

In [37]:
def data_cleaning(tX):
    
    length = len(tX)
    dim = tX.shape[1]
    delete_index=np.array([],dtype=np.int32)
    

    #for all features
    for d in range(dim):
        if(np.abs(np.std(tX[:,d]))<1e-4):
            delete_index = np.append(delete_index,d)
        else:
            median = np.median(tX[:,d][tX[:,d]!=-999])
            tX[:,d][tX[:,d]==-999] = median
            mean = np.mean(tX[:,d])
            std = np.std(tX[:,d])
            _max = mean+2*std
            _min = mean-2*std
            
            tX[:,d][tX[:,d]>_max] = _max
            tX[:,d][tX[:,d]<_min] = _min
            
            #calculate again
            tX[:,d] = (tX[:,d]-np.mean(tX[:,d]))/np.std(tX[:,d])
        
    tX=np.delete(tX,delete_index,axis=1)
    return tX

In [57]:
def split_groups(y_train,y_test,tX_train,tX_test):
#     tX_train_jet_num = tX_train[:,22]
#     tX_test_jet_num = tX_test[:,22]
    tX_train_jet_num = tX_train[:,9]
    tX_test_jet_num = tX_test[:,9]
    
    y_train_0 = np.copy(y_train[tX_train_jet_num==0])
    y_train_1 = np.copy(y_train[tX_train_jet_num==1])
    y_train_2 = np.copy(y_train[tX_train_jet_num==2])
    y_train_3 = np.copy(y_train[tX_train_jet_num==3])
    
    y_test_0 = np.copy(y_test[tX_test_jet_num==0])
    y_test_1 = np.copy(y_test[tX_test_jet_num==1])
    y_test_2 = np.copy(y_test[tX_test_jet_num==2])
    y_test_3 = np.copy(y_test[tX_test_jet_num==3])
    
    tX_train_0 =np.copy(tX_train[tX_train_jet_num==0])
    tX_train_1 =np.copy(tX_train[tX_train_jet_num==1])
    tX_train_2 =np.copy(tX_train[tX_train_jet_num==2])
    tX_train_3 =np.copy(tX_train[tX_train_jet_num==3])
    
    tX_test_0 = np.copy(tX_test[tX_test_jet_num==0])
    tX_test_1 = np.copy(tX_test[tX_test_jet_num==1])
    tX_test_2 = np.copy(tX_test[tX_test_jet_num==2])
    tX_test_3 = np.copy(tX_test[tX_test_jet_num==3])
    
    return y_train_0,y_train_1,y_train_2,y_train_3,y_test_0,y_test_1,y_test_2,y_test_3,tX_train_0,tX_train_1,tX_train_2,tX_train_3,tX_test_0,tX_test_1,tX_test_2,tX_test_3


In [58]:
def build_poly(tX,degree,dim):
    
    N=tX.shape[0]
    
    
    for d in range(dim):
        phi = np.zeros((N,degree-1))
        valid_index = (tX[:,d]!=-999)
        for t in range(2,degree+1):
            phi[valid_index,t-2] = np.power(tX[valid_index,d],t)
            phi[~valid_index,t-2] = -999
        tX = np.concatenate((tX,phi),axis=1)
    return tX

def build_sqrt(tX,dim):
    N = tX.shape[0]
    phi = np.zeros((N,30))
    
    
    for d in range(dim):
        valid_index = ((tX[:,d]>=0)&(tX[:,d]!=-999))
        phi[valid_index,d] = np.sqrt(tX[valid_index,d])
        phi[~valid_index,d] = 0
    
    tX = np.concatenate((tX,phi),axis=1)
    
    return tX

def build_log(tX,dim):
    N = tX.shape[0]
    phi = np.zeros((N,30))
    
    for d in range(dim):
        valid_index = ((tX[:,d]>0)&(tX[:,d]!=-999))
        phi[valid_index,d] = np.log(tX[valid_index,d])
        phi[~valid_index,d] = 0
    
    tX = np.concatenate((tX,phi),axis=1)
    
    return tX

def build_cos_sin(tX,dim):
    N = tX.shape[0]
    phi_cos = np.zeros((N,30))
    phi_sin = np.zeros((N,30))
    
    for d in range(dim):
        valid_index = (tX[:,d]!=-999)
        phi_cos[valid_index,d] = np.cos(tX[valid_index,d])
        phi_sin[valid_index,d] = np.sin(tX[valid_index,d])
    
    tX = np.concatenate((tX,phi_cos),axis=1)
    tX = np.concatenate((tX,phi_sin),axis=1)
    
    return tX

def build_cross_term(tX,dim):
    N = tX.shape[0]
    phi = np.zeros((N,int(30*30)))
    t = 0
    
    for d1 in range(dim):
        for d2 in range(dim):
            valid_index = ((tX[:,d1]!=-999)&(tX[:,d2]!=-999))
            phi[valid_index,t] = tX[valid_index,d1]*tX[valid_index,d2]
            t = t+1
    
    tX = np.concatenate((tX,phi),axis=1)
            
    return tX

In [59]:
def build_feature(tX,degree):
    
    dim = tX.shape[1]
    tX=build_poly(tX,degree,dim)
    tX=build_sqrt(tX,dim)
    tX=build_log(tX,dim)
    tX=build_cross_term(tX,dim)
    
    return tX

In [60]:
def build_k_indices(y,k_fold,seed):
    num_row = y.shape[0]
    interval = int(num_row/k_fold)
    np.random.seed(seed)
    indices=np.random.permutation(num_row)
    k_indices = [indices[k*interval:(k+1)*interval] for k in range (k_fold)]
    return np.array(k_indices)

In [61]:
def cross_validation(y,tX,k_indices,k,lambda_,gamma,max_iters):
    
    tX_test_indice = k_indices[k]
    tX_train_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tX_train_indice = tX_train_indice.reshape(-1)
    y_test = y[tX_test_indice]
    y_train = y[tX_train_indice]
    tX_test = tX[tX_test_indice]
    tX_train = tX[tX_train_indice]
    
    #split train and test into four groups
    y_train_0,y_train_1,y_train_2,y_train_3,y_test_0,y_test_1,y_test_2,y_test_3,tX_train_0,tX_train_1,tX_train_2,tX_train_3,tX_test_0,tX_test_1,tX_test_2,tX_test_3=split_groups(y_train,y_test,tX_train,tX_test)
    
    tX_train_0=data_cleaning(tX_train_0)
    tX_train_1=data_cleaning(tX_train_1)
    tX_train_2=data_cleaning(tX_train_2)
    tX_train_3=data_cleaning(tX_train_3)
    
    tX_test_0=data_cleaning(tX_test_0)
    tX_test_1=data_cleaning(tX_test_1)
    tX_test_2=data_cleaning(tX_test_2)
    tX_test_3=data_cleaning(tX_test_3)  

#     tX_train_0 = build_feature(tX_train_0,3)
#     tX_train_1 = build_feature(tX_train_1,3)
#     tX_train_2 = build_feature(tX_train_2,3)
#     tX_train_3 = build_feature(tX_train_3,3)
    

    
#     tX_test_0 = build_feature(tX_test_0,3)
#     tX_test_1 = build_feature(tX_test_1,3)
#     tX_test_2 = build_feature(tX_test_2,3)
#     tX_test_3 = build_feature(tX_test_3,3)
    
#     tX_train_0=data_cleaning(tX_train_0)
#     tX_train_1=data_cleaning(tX_train_1)
#     tX_train_2=data_cleaning(tX_train_2)
#     tX_train_3=data_cleaning(tX_train_3)
    
#     tX_test_0=data_cleaning(tX_test_0)
#     tX_test_1=data_cleaning(tX_test_1)
#     tX_test_2=data_cleaning(tX_test_2)
#     tX_test_3=data_cleaning(tX_test_3)      
    
#     weight_0,y_train_pred_0=stacking_train(y_train_0,tX_train_0,lambda_,gamma,max_iters)
#     y_test_pred_0=stacking_test(tX_test_0,weight_0)
    
#     weight_1,y_train_pred_1=stacking_train(y_train_1,tX_train_1,lambda_,gamma,max_iters)
#     y_test_pred_1=stacking_test(tX_test_1,weight_1)
    
#     weight_2,y_train_pred_2=stacking_train(y_train_2,tX_train_2,lambda_,gamma,max_iters)
#     y_test_pred_2=stacking_test(tX_test_2,weight_2)
    
#     weight_3,y_train_pred_3=stacking_train(y_train_3,tX_train_3,lambda_,gamma,max_iters)
#     y_test_pred_3=stacking_test(tX_test_3,weight_3)
    
#     w_0,cost_0 = ridge_regression(y_train_0,y_train_pred_0,lambda_)
#     w_1,cost_1 = ridge_regression(y_train_1,y_train_pred_1,lambda_)
#     w_2,cost_2 = ridge_regression(y_train_2,y_train_pred_2,lambda_)
#     w_3,cost_3 = ridge_regression(y_train_3,y_train_pred_3,lambda_)
    
#     y_pred_0 = predict_labels(w_0, y_test_pred_0)
#     y_pred_1 = predict_labels(w_1, y_test_pred_1)
#     y_pred_2 = predict_labels(w_2, y_test_pred_2)
#     y_pred_3 = predict_labels(w_3, y_test_pred_3)

    y_pred_0 = random_forest(y_train_0,tX_train_0,tX_test_0,lambda_,100,max_iters,gamma)
    y_pred_1 = random_forest(y_train_1,tX_train_1,tX_test_1,lambda_,100,max_iters,gamma)
    y_pred_2 = random_forest(y_train_2,tX_train_2,tX_test_2,lambda_,100,max_iters,gamma)
    y_pred_3 = random_forest(y_train_3,tX_train_3,tX_test_3,lambda_,100,max_iters,gamma)
    

    
    num_0 = np.count_nonzero(y_pred_0==y_test_0)
    num_1 = np.count_nonzero(y_pred_1==y_test_1)
    num_2 = np.count_nonzero(y_pred_2==y_test_2)
    num_3 = np.count_nonzero(y_pred_3==y_test_3)
    
    accuracy_0 = num_0/len(y_pred_0)
    accuracy_1 = num_1/len(y_pred_1)
    accuracy_2 = num_2/len(y_pred_2)
    accuracy_3 = num_3/len(y_pred_3)
    
    num = len(y_test)
    print(f'group 0 accuracy: {accuracy_0}')
    print(f'group 1 accuracy: {accuracy_1}')
    print(f'group 2 accuracy: {accuracy_2}')
    print(f'group 3 accuracy: {accuracy_3}')
    accuracy = (num_0+num_1+num_2+num_3)/num
    
    return accuracy




In [62]:
def stacking_train(y,tX,lambda_,gamma,max_iters):
    
    N = tX.shape[0]
    dim = tX.shape[1]
    initial_w = np.zeros(dim)
    weight = np.zeros((3,dim))
    y_pred = np.zeros((N,3))
    
    #least squares
    weight[0,:],cost = least_squares(y, tX)
    y_pred[:,0]=predict_labels(weight[0,:].T, tX)


    
    #reg Logistic Regression
    
    weight[1,:],cost_LR= reg_logistic_regression(y, tX, lambda_, initial_w, max_iters, gamma)
    y_pred[:,1]=predict_labels(weight[1,:], tX)

    
    #Ridge Regression
    weight[2,:],cost_RR = ridge_regression(y,tX,lambda_)
    y_pred[:,2]=predict_labels(weight[2,:],tX)
    
    
    return weight,y_pred

In [63]:
def stacking_test(tX,weight):
    
    N =tX.shape[0]
    y_pred = np.zeros((N,3))
    
    y_pred[:,0]=predict_labels(weight[0,:], tX)    
    y_pred[:,1]=predict_labels(weight[1,:], tX)    
    y_pred[:,2]=predict_labels(weight[2,:], tX)

    
    return y_pred

In [64]:
def random_forest(y_train,tX_train,tX_test,lambda_,max_num,max_iters,gamma):
    
    N_test= tX_test.shape[0]
    N_train = tX_train.shape[0]
    dim = tX_test.shape[1]
    y_pred_total_train = np.zeros((N_train,max_num))
    y_pred_total_test = np.zeros((N_test,max_num))

    
    
    for t in range(max_num):
        #generate an random feature index
        feature_index=np.random.choice(range(0,dim),int((dim)/3))
        initial_w = np.zeros(int((dim)/3))
        #feature_index_m=np.concatenate((feature_index,np.array(range(0,30))))

        #w_pred,cost = reg_logistic_regression(y_train, tX_train[:,feature_index_m], lambda_, initial_w, max_iters, gamma)
        w_pred,cost = ridge_regression(y_train,tX_train[:,feature_index],lambda_)
        y_pred_total_train[:,t] = predict_labels(w_pred,tX_train[:,feature_index])
        y_pred_total_test[:,t] = predict_labels(w_pred,tX_test[:,feature_index])
    #combine the answers
    w,cost_ = ridge_regression(y_train,y_pred_total_train,lambda_)
    y_pred = predict_labels(w,y_pred_total_test)
    return y_pred

In [65]:
def demo():
    k_fold = 4
    seed = 12
    k_indices = build_k_indices(y, k_fold, seed)
    total = 0
    
    for k in range(k_fold):
        accuracy = cross_validation(y,tX,k_indices,k,1e-9,1e-9,10000)
        total  = total+accuracy
        print(f'{k}:{accuracy} ')
    
    average=total/k_fold
    print(f'average accuracy:{average}')

In [66]:
demo()

group 0 accuracy: 0.7229305727646989
group 1 accuracy: 0.6570092977859968
group 2 accuracy: 0.6135033277203111
group 3 accuracy: 0.6427927115280534
0:0.673456 
group 0 accuracy: 0.7293329077561442
group 1 accuracy: 0.6481720208322591
group 2 accuracy: 0.6131404174573055
group 3 accuracy: 0.6426320667284523
1:0.673152 
group 0 accuracy: 0.7143372046455747
group 1 accuracy: 0.6530029093931837
group 2 accuracy: 0.6265231840481089
group 3 accuracy: 0.6330616583982991
2:0.670352 
group 0 accuracy: 0.7157683024939662
group 1 accuracy: 0.6576456060917885
group 2 accuracy: 0.6181270797021075
group 3 accuracy: 0.6345395915442493
3:0.67072 
average accuracy:0.6719200000000001
